#### TELECOM CUSTOMER CHURN : DATA 428 PROJECT
# 2. Descriptive Analysis and Data Manipulation

### GENERAL NOTES ABOUT THE DATA AND JUPYTER NOTEBOOKS USED IN THE PROJECT

- The ‘Telecom Customer’ dataset contains data for 100,000 customers with 99 variables collected 
over five years (max months of customer is 61). 
- It has been downloaded from ‘https://www.kaggle.com/abhinav89/telecom-customer’. 
- Original file name is "Telecom Customer.csv" - contains difficult to decipher column names.

- There are a total of five Jupyter Notebooks for ease of keeping the different approaches seperate and allow room for explorations within each approach
- Notebook No 1   : Exploratory Analysis
- Notebook No 2   : Descriptive Analysis and DATA MANIPULATION (Flow for Imputation & Dummy Variables)
- Notebook No 3   : Approach - 1 for Model Building (using Log & Yeo-Johnson Transformation)
- Notebook No 3(B): To create and save Yeo-Johnson transformed data. Kept separate as done on another machine and bring in the data. (PowerTransformer is only available in versions 0.20.0 of Sci-Kit Learn)
- Notebook No 4   : Approach - 2 for Model Building using the 'Pipeline' method, based on log transformed data. The notebook has an option of choosing Yeo-Johnson data, but for ready reference, Notebbok 4(B) was made.
- Notebook No 4(B): Approach - 2 with Yeo_Johnson transformed data.(Notebook is same as No. 4.
- Notebook No 5   : testing the effects of reduced features based on CONTINUITY FACTOR. A separate flow to test the effects using best Random Forest and ADA Boost models.

- Jotinder S Sudan, 15 March 2019, University of Canterbury, Christchurch
    
### This Notebook :  No. 2
 
- Dropping some redundant columns and outliers (only one observation)
- Descriptive Analysis - examining 95th & 99th quantiles with respect to extremely high values. 
- Data Manipulation : Preparing different versions of the dataset (all were not used ahead)

    - v1 : all blanks removed : Later, the models did not give satisfactory results on 26,000 observations.

    - v2 : all blanks removed after removing redundant columns and outliers (results - same as above)

    - v3 :
         - categories with poor representation (less than 5%), clubbed into 'others'.
         - 'U' values - treated as 'missing value'. dropped in case higher in volume or imputed with mode

    - v4 : Creating a dataset with extreme high values (beyond 5xIQR) by 99th quantile (Flow created - NOT used)

In [43]:
import pandas as pd
import numpy as np
import scipy as sp 
import scipy.stats as sps
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [44]:
# Using dataset with better column names

data = pd.read_csv("telecom_data_better_colnames.csv")

In [45]:
data.drop('Unnamed: 0', axis=1, inplace=True)

In [46]:
data.shape

(100000, 100)

In [47]:
# Checking the data types
#data.dtypes

## Data Wrangling - following issues to be addressed
    - 'geographic_area' : names can be made better to avoid errors (learnt after making mistakes initially)
    - drop rows with outliers 
    - drop columns with very high 'nan' / blanks
    - drop columns like 'child_between_0_to_2_years_in_house' 

In [48]:
data["geogrpahic_area"].unique()

array(['NORTHWEST/ROCKY MOUNTAIN AREA', 'CHICAGO AREA',
       'GREAT LAKES AREA', 'NEW ENGLAND AREA', 'DALLAS AREA',
       'CENTRAL/SOUTH TEXAS AREA', 'TENNESSEE AREA', 'MIDWEST AREA',
       'PHILADELPHIA AREA', 'OHIO AREA', 'HOUSTON AREA', 'SOUTHWEST AREA',
       'NEW YORK CITY AREA', 'ATLANTIC SOUTH AREA', 'SOUTH FLORIDA AREA',
       'CALIFORNIA NORTH AREA', 'DC/MARYLAND/VIRGINIA AREA',
       'NORTH FLORIDA AREA', nan, 'LOS ANGELES AREA'], dtype=object)

In [49]:
geo_area = []
for i in range(len(data)):
    item = data["geogrpahic_area"][i]
    #print(item)
    if type(item) is float:
        geo_area.append(item)
        #print(item)
    else:
        name = item.replace(' ', '_')
        name = name.replace('/', '_')
        geo_area.append(name)

In [50]:
data.drop("geogrpahic_area", inplace=True, axis=1)

In [51]:
data["geogrpahic_area"] = geo_area

In [52]:
data["geogrpahic_area"].unique()

array(['NORTHWEST_ROCKY_MOUNTAIN_AREA', 'CHICAGO_AREA',
       'GREAT_LAKES_AREA', 'NEW_ENGLAND_AREA', 'DALLAS_AREA',
       'CENTRAL_SOUTH_TEXAS_AREA', 'TENNESSEE_AREA', 'MIDWEST_AREA',
       'PHILADELPHIA_AREA', 'OHIO_AREA', 'HOUSTON_AREA', 'SOUTHWEST_AREA',
       'NEW_YORK_CITY_AREA', 'ATLANTIC_SOUTH_AREA', 'SOUTH_FLORIDA_AREA',
       'CALIFORNIA_NORTH_AREA', 'DC_MARYLAND_VIRGINIA_AREA',
       'NORTH_FLORIDA_AREA', nan, 'LOS_ANGELES_AREA'], dtype=object)

In [53]:
data.shape

(100000, 100)

### Creating a dataset by deleting all 'nan' / 'blanks'  --> original_data_without_NA.csv

In [54]:
original_data_without_NA = data.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)

In [55]:
original_data_without_NA = original_data_without_NA.reset_index()
original_data_without_NA.drop('index', axis = 1, inplace=True) 

In [56]:
original_data_without_NA.shape

(26704, 100)

In [57]:
# Saving to a file

#original_data_without_NA.to_csv('original_data_without_NA.csv')

#### no_of_unique_subscribers_in_the_house 
    - This feature has numeric discrete values and some outliers like 196 unique users in a house.
    - It's either a wrong entry or a 'close user group' (CUG) scenario of an organisation. 
    - Hence, dropping any entry with more than 20 subscribers in the house (making concessions for a hostel)

In [58]:
data['no_of_unique_subscribers_in_the_house'].unique()

array([  2,   1,   3,   5,   4,   6,   7,   9,   8,  12,  11,  10,  13,
        18, 196], dtype=int64)

In [59]:
for i in range(len(data)): 
    
    value = data['no_of_unique_subscribers_in_the_house'][i]
       
    if value > 20:
        data.drop(data.index[i], inplace = True)   
data = data.reset_index()
data.drop('index', axis = 1, inplace=True)

#         no_of_unique_subscribers.append('low')
#     elif value > 2 and value < 7:
#         no_of_unique_subscribers.append('medium')
#     else:
#         no_of_unique_subscribers.append('high')

In [60]:
for i in range(len(data)): 
    
    value_2 = data['no_of_active_subscribers_in_house'][i]
       
    if value_2 > 20:
        data.drop(data.index[i], inplace = True)    
        
data = data.reset_index()
data.drop('index', axis = 1, inplace=True)   

In [61]:
data.shape

(99999, 100)

### Dropping features with large missing values
    - 'known_no_of_vehicles' - 49.3% values missing (refer the data exploration notebook)

In [62]:
data.drop('known_no_of_vehicles', inplace=True, axis = 1)

In [63]:
data.drop('cust_id', inplace=True, axis = 1)

In [64]:
data.shape

(99999, 98)

### FEATURES WITH 'U'  : Assuming 'U' to be 'Unknown' or 'Missing Value'

    - new_cell_phone_user : 66.9% 'U' values, which are as good as missing values
    - dualband :  "Y", "N", "T", "U".  Where 'U' is only 0.02%

In [65]:
data.new_cell_phone_user.value_counts()

U    66913
Y    19301
N    13785
Name: new_cell_phone_user, dtype: int64

In [66]:
data.drop('new_cell_phone_user', inplace=True, axis = 1)

In [67]:
# Need to decide - what to do with 'U'
data.dualband.value_counts()

Y    72264
N    23196
T     4316
U      222
Name: dualband, dtype: int64

In [68]:
data['dualband'] = data['dualband'].str.replace('U','Y')

In [69]:
# need to decide what to do with 'UNKW'
data.handset_web_capability.value_counts()

WCMB    75732
WC      13843
UNKW      235
Name: handset_web_capability, dtype: int64

In [70]:
data['handset_web_capability'] = data['handset_web_capability'].str.replace('UNKW', 'WCMB')

In [71]:
data.handset_web_capability.value_counts()

WCMB    75967
WC      13843
Name: handset_web_capability, dtype: int64

In [72]:
# Leaving 'U' as such because no information is available about the categories.
data.marital_status.value_counts()

U    37332
M    31052
S    17627
B     7116
A     5140
Name: marital_status, dtype: int64

### Dropping five columns of 'children in house' as data very sparse and not clear

In [73]:
list_of_children = ['child_0_to_2_yrs_in_house', 'child_3_to_5_yrs_in_house',
                   'child_6_to_10_yrs_in_house', 'child_11_to_15_yrs_in_house',
                   'child_16_to_17_yrs_in_house']

In [74]:
#redundant_unknown_features = ["truck_indicator", "rv_indicator", "foreign_travel_dummy_variable", "infobase_match"]

In [75]:
def drop_feature(drop_list):
    for feature in drop_list:
        data.drop(feature, inplace=True, axis = 1)

In [76]:
drop_feature(list_of_children)

In [77]:
#drop_feature(redundant_unknown_features)

In [78]:
data.shape

(99999, 92)

### Now that we have removed some redundant columns, lets remove all 'nan' and create another dataset for analysis - dataset_without_NA_2.csv

In [79]:
dataset_without_NA_2 = data.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)

In [80]:
dataset_without_NA_2 = dataset_without_NA_2.reset_index()
dataset_without_NA_2.drop('index', axis = 1, inplace=True)

In [81]:
dataset_without_NA_2.shape

(36132, 92)

In [82]:
# Saving to a file

#dataset_without_NA_2.to_csv("dataset_without_NA_2.csv")

### Moving towards Separating columns which appear numeric (continuous), but are actually factors or discrete numbers

In [83]:
numeric_df = data.corr()

In [356]:
list_numeric_cols = list(numeric_df.columns)
data[list_numeric_cols].describe().T

,count,mean,std,min,25%,50%,75%,max
avg_mthly_rev,99642.0,58.719894,46.291901,-6.167500,33.260000,48.195000,70.750000,3843.262500
avg_no_of_mthly_mins,99642.0,513.565091,525.168255,0.000000,150.750000,355.625000,703.000000,12206.750000
avg_total_mthly_recurring_charge,99642.0,46.178746,23.623288,-26.915000,30.000000,44.990000,59.990000,409.990000
avg_no_of_directory_assisted_calls,99642.0,0.888837,2.177628,0.000000,0.000000,0.247500,0.990000,159.390000
avg_overage_mins,99642.0,41.072659,97.296552,0.000000,0.000000,2.750000,42.000000,4320.750000
avg_overage_rev,99642.0,13.559696,30.501008,0.000000,0.000000,1.000000,14.437500,1102.400000
avg_rev_of_voice_overage,99642.0,13.295195,30.056211,0.000000,0.000000,0.682500,14.025000,896.087500
avg_rev_of_data_overage,99642.0,0.261320,3.126547,0.000000,0.000000,0.000000,0.000000,423.540000
avg_no_of_roaming_calls,99642.0,1.286418,14.711447,0.000000,0.000000,0.000000,0.235000,3685.200000
percntg_change_in_mthly_mins_vs_previous_three_month_avg,99108.0,-13.933959,276.088898,-3875.000000,-87.000000,-6.250000,63.000000,31219.250000


### Complete List of numeric features which are not continuous

    - List used for imputing with 'mode' for these features

In [85]:
list_of_numeric_factors = ["no_of_unique_subscribers_in_the_house",
                           "no_of_active_subscribers_in_house",
                           "no_of_handsets_issued",
                           "no_of_models_issued",
                           "no_of_adults_in_house",  
                           "estimated_income",
                           "length_of_residence", 
                           "truck_indicator",
                           "rv_indicator",
                           "foreign_travel_dummy_variable"]

In [86]:
numeric_features_set = set(list_numeric_cols) - set(list_of_numeric_factors)
numeric_features = list(numeric_features_set)
len(numeric_features)

67

In [87]:
categorical_features_set = set(list(data.columns)) - set(list_numeric_cols)
categorical_features = list(categorical_features_set)
len(categorical_features)

15

In [88]:
cat_required = ["AA", "A", "BA", "CA", "EA"]
cat_with_others = []

for i in range(len(data)):
    cat = data.credit_class_code[i]
    
    if cat in cat_required:
        cat_with_others.append(cat)
        
    if cat not in cat_required:
        cat_with_others.append("others")
    

In [89]:
data.credit_class_code = cat_with_others

In [90]:
ethnicity_code = []
ethnicity_code_required = ["X", "P", "M", "R", "D", "B", "F"]

for i in range(len(data)):
    cat = data.ethnicity_roll_to_up_code[i]
    
    if cat in ethnicity_code_required:
        ethnicity_code.append("others")
        
    if cat not in ethnicity_code_required:
        ethnicity_code.append(cat)
        

In [91]:
data['ethnicity_roll_to_up_code'] = ethnicity_code

In [92]:
data['ethnicity_roll_to_up_code'].unique()

array(['N', 'Z', 'U', 'I', 'S', 'others', 'J', 'H', 'G', 'O', nan, 'C'],
      dtype=object)

In [93]:
data.credit_class_code.unique()

array(['A', 'EA', 'others', 'BA', 'CA', 'AA'], dtype=object)

In [357]:
data.shape

(99999, 92)

In [95]:
data.to_csv('data_before_log.csv')

## MANIPULATING NUMERIC FEATURES - Looking at Outliers, using IQR and 99th quantile values : but not used

In [360]:
# first_quart = []
# third_quart = []
# boundary = []
# ninetyfive_quant = []
# ninetynine_quant = []
# original_max = []


# for features in numeric_features:
#     first = data[features].quantile(0.25)
    
#     #first_quart.append(first)
    
#     third = data[features].quantile(.75)
#     #third_quart.append(third)

#     #Could change boundary if required from IQR x 1.5 to 3 or 5 as per the situation

#     boundary_val = round((third + (third - first) * 3),2)   
#     #print(boundary_val)
#     boundary.append(boundary_val)
#     ninetyfive_quant.append(data[features].quantile(.95))
#     ninetynine_quant.append(data[features].quantile(.99))
#     original_max.append(data[features].max())

In [362]:
#describe_df = data[numeric_features].describe().T

In [363]:
#describe_df["boundary"] = boundary

In [364]:
#describe_df["95_quant"] = ninetyfive_quant

In [365]:
#describe_df["99_quant"] = ninetynine_quant

In [366]:
#describe_df["original_max"] = original_max

In [367]:
#describe_df.T

,avg_no_of_peak_data_calls,avg_no_of_attempted_data_calls_placed,billing_adjusted_total_no_of_calls_over_the_life_of_the_cust,avg_rev_of_voice_overage,avg_no_of_completed_data_calls,avg_total_mthly_recurring_charge,avg_rounded_mins_of_cust_care_calls,total_no_of_calls_over_the_life_of_the_cust,avg_no_of_unans_data_calls,churn,...,avg_no_of_completed_calls,no_of_days_of_current_equipment,avg_unrounded_mins_of_off_to_peak_data_calls,avg_mthly_rev_over_the_previous_three_months,avg_mthly_no_of_calls_over_the_previous_six_months,avg_no_of_attempted_calls,avg_no_of_three_way_calls,avg_unrounded_mins_of_completed_voice_calls,avg_overage_rev,avg_no_of_call_forwarding_calls
count,99999.000000,99999.000000,99999.000000,99642.000000,99999.000000,99642.000000,99999.000000,99999.000000,99999.000000,99999.000000,...,99999.000000,99997.000000,99999.000000,99999.000000,97160.000000,99999.000000,99999.000000,99999.000000,99642.000000,99999.000000
mean,0.358165,0.870557,2836.384514,13.295195,0.774449,46.178746,4.666927,2877.159932,0.029810,0.495615,...,109.668073,391.932238,1.135815,59.193242,178.366828,145.753683,0.284723,227.765797,13.559696,0.011700
std,4.065650,9.053946,3756.528545,30.056211,8.129884,23.623288,12.762008,3790.878155,0.496973,0.499983,...,119.594400,256.484711,17.768772,46.695654,182.724763,159.348224,1.092106,264.404235,30.501008,0.547473
min,0.000000,0.000000,0.000000,0.000000,0.000000,-26.915000,0.000000,0.000000,0.000000,0.000000,...,0.000000,-5.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,868.000000,0.000000,0.000000,30.000000,0.000000,889.000000,0.000000,0.000000,...,28.666667,212.000000,0.000000,33.000000,59.000000,38.333333,0.000000,49.053333,0.000000,0.000000
50%,0.000000,0.000000,1789.000000,0.682500,0.000000,44.990000,0.000000,1822.000000,0.000000,0.000000,...,76.000000,342.000000,0.000000,48.000000,127.000000,101.000000,0.000000,146.203333,1.000000,0.000000
75%,0.000000,0.000000,3442.000000,14.025000,0.000000,59.990000,4.000000,3492.000000,0.000000,1.000000,...,150.666667,530.000000,0.000000,71.000000,237.000000,199.666667,0.333333,309.478333,14.437500,0.000000
max,281.000000,733.666667,98705.000000,896.087500,559.333333,409.990000,861.333333,98874.000000,81.666667,1.000000,...,1894.333333,1823.000000,2922.043333,1593.000000,3256.000000,2289.000000,66.000000,4514.453333,1102.400000,81.333333
boundary,0.000000,0.000000,11164.000000,56.100000,0.000000,149.960000,16.000000,11301.000000,0.000000,4.000000,...,516.670000,1484.000000,0.000000,185.000000,771.000000,683.670000,1.330000,1090.750000,57.750000,0.000000
95_quant,0.666667,1.666667,8592.100000,64.349375,1.333333,84.990000,23.666667,8701.100000,0.000000,1.000000,...,334.000000,884.000000,1.083333,139.000000,514.000000,445.000000,1.333333,743.564333,64.899750,0.000000


In [104]:
#describe_df_2 = describe_df.T

In [105]:
#describe_df_2.drop('churn', inplace = True, axis = 1)

In [106]:
# for feature in describe_df_2:
#     if describe_df_2[feature]['99_quant'] < describe_df_2[feature]['boundary']:
#         print(feature)

In [107]:
#describe_df_2.drop('current_handset_price', inplace = True, axis = 1)

In [108]:
# Creating a new dataset with 99 quantile for all outliers

# for feature in describe_df_2:
#     new_data = []
#     quant_99 = describe_df_2[feature]['99_quant']
    
#     for i in range(len(data)):
#         if data[feature][i] < quant_99:
#             new_data.append(data[feature][i])
#         else:
#             new_data.append(quant_99)
            
#     data[feature] = new_data  
    

In [109]:
#data.describe()

### Removing all missing values from the dataset at this stage to create 

In [110]:
#data.shape

In [111]:
#quant99_data_without_NA_3 = data.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)

In [112]:
# quant99_data_without_NA_3 = quant99_data_without_NA_3.reset_index()
# quant99_data_without_NA_3.drop('index', axis = 1, inplace=True) 

In [113]:
#quant99_data_without_NA_3.shape

In [114]:
#quant99_data_without_NA_3.to_csv('quant99_data_without_NA_3.csv')

### Notebook 2 Ends Here